# CSCI 331 - Homework 3 - Rahat Moktadir

## Proposition 1: Top 10 Customers with Year-Over-Year Growth

**Functional Specification:**

- **Objective:** To identify the top 10 customers by sales in the most recent full year and calculate their sales growth compared to the prior year.
- **End-User:** Strategic Account Managers and the Sales department.
- **Business Value:** This query finds our most valuable \*and\* fastest-growing customers. This insight is critical for allocating resources to nurture key accounts and drive revenue growth.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [16]:
-- Step 1: Declare variables to hold the years we want to analyze.
DECLARE @RecentYear INT = (SELECT MAX([Calendar Year]) FROM Dimension.Date WHERE [Date] < GETDATE());
DECLARE @PriorYear INT = @RecentYear - 1;

-- Step 2: Use the variables in the main query.
WITH SalesByYear AS (
    SELECT
        c.Customer,
        d.[Calendar Year],
        SUM(s.[Total Including Tax]) AS TotalSales
    FROM Fact.Sale s
    JOIN Dimension.Customer c ON s.[Customer Key] = c.[Customer Key]
    JOIN Dimension.Date d ON s.[Invoice Date Key] = d.[Date]
    WHERE d.[Calendar Year] IN (@RecentYear, @PriorYear)
    GROUP BY c.Customer, d.[Calendar Year]
),
CustomerGrowth AS (
    SELECT
        Customer,
        MAX(CASE WHEN [Calendar Year] = @RecentYear THEN TotalSales ELSE 0 END) AS RecentYearSales,
        MAX(CASE WHEN [Calendar Year] = @PriorYear THEN TotalSales ELSE 0 END) AS PriorYearSales
    FROM SalesByYear
    GROUP BY Customer
)
SELECT TOP 10
    Customer,
    RecentYearSales,
    PriorYearSales,
    CAST(((RecentYearSales - PriorYearSales) / PriorYearSales) * 100 AS DECIMAL(10, 2)) AS GrowthPercentage
FROM CustomerGrowth
WHERE PriorYearSales > 0
ORDER BY RecentYearSales DESC;

(10 rows affected)

Total execution time: 00:00:01.296

Customer,RecentYearSales,PriorYearSales,GrowthPercentage
Unknown,10224728.14,23971428.29,-57.35
"Tailspin Toys (Arietta, NY)",100315.81,88846.94,12.91
"Tailspin Toys (Good Hart, MI)",98938.98,97622.81,1.35
"Wingtip Toys (Obetz, OH)",98399.17,98937.51,-0.54
"Wingtip Toys (North Beach Haven, NJ)",96196.45,94908.12,1.36
"Wingtip Toys (Leathersville, GA)",89085.91,96527.73,-7.71
"Tailspin Toys (Inguadona, MN)",85723.90,97813.78,-12.36
"Wingtip Toys (Cale, AR)",84442.95,86542.06,-2.43
"Wingtip Toys (Ware Shoals, SC)",82556.27,75012.34,10.06
"Wingtip Toys (Bourneville, OH)",82183.30,102381.59,-19.73


## <span style="color: var(--vscode-foreground);">Proposition 2: Top 5 Products by Monthly Sales Rank</span>

**Functional Specification:**

- **Objective:** To identify the top 5 best-selling products by quantity for each month of the last year, showing how product popularity shifts over time.
- **End-User:** Marketing and Inventory Management teams.
- **Business Value:** This reveals product seasonality and trends, allowing for smarter inventory planning and targeted marketing campaigns during peak months.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [13]:
WITH MonthlySales AS (
    SELECT
        d.[Calendar Month Label] AS SaleMonth,
        si.[Stock Item],
        SUM(s.Quantity) AS TotalQuantity,
        DENSE_RANK() OVER(PARTITION BY d.[Calendar Month Label] ORDER BY SUM(s.Quantity) DESC) AS Rank
    FROM Fact.Sale s
    JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
    JOIN Dimension.Date d ON s.[Invoice Date Key] = d.[Date]
    WHERE d.[Calendar Year] = (SELECT MAX([Calendar Year]) FROM Dimension.Date WHERE [Date] < GETDATE())
    GROUP BY d.[Calendar Month Label], si.[Stock Item]
)
SELECT
    SaleMonth,
    [Stock Item],
    TotalQuantity,
    Rank
FROM MonthlySales
WHERE Rank <= 5
ORDER BY SaleMonth, Rank;

(25 rows affected)

Total execution time: 00:00:00.188

SaleMonth,Stock Item,TotalQuantity,Rank
CY2016-Apr,Shipping carton (Brown) 413x285x187mm,6900,1
CY2016-Apr,Black and orange fragile despatch tape 48mmx75m,6336,2
CY2016-Apr,Red and white urgent despatch tape 48mmx75m,5880,3
CY2016-Apr,Clear packaging tape 48mmx75m,5070,4
CY2016-Apr,Chocolate echidnas 250g,4488,5
CY2016-Feb,Black and orange handle with care despatch tape 48mmx100m,4944,1
CY2016-Feb,Black and orange fragile despatch tape 48mmx75m,4752,2
CY2016-Feb,Shipping carton (Brown) 356x356x279mm,4350,3
CY2016-Feb,Shipping carton (Brown) 229x229x229mm,4250,4
CY2016-Feb,Chocolate beetles 250g,4224,5


## <span style="color: var(--vscode-foreground);">Proposition 3: Sales Team Performance Against Company Average</span>

Functional Specification:

- **Objective:** To evaluate each salesperson's performance by comparing their average sale profit to the company's overall average sale profit.
- **End-User:** Sales VPs and HR.
- **Business Value:** Provides a fair, data-driven benchmark for employee performance reviews, helping to identify top performers for bonuses and team members who may need additional training.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [3]:
-- Step 1: Calculate the overall average profit per sale across the entire company.
DECLARE @CompanyAverageProfit DECIMAL(18, 2) = (SELECT AVG(Profit) FROM Fact.Sale);

-- Step 2: Calculate each salesperson's metrics and compare to the company average.
SELECT
    e.Employee,
    COUNT(s.[Sale Key]) AS NumberOfSales,
    AVG(s.Profit) AS EmployeeAverageProfit,
    @CompanyAverageProfit AS CompanyAverageProfit,
    (AVG(s.Profit) - @CompanyAverageProfit) AS Variance
FROM Fact.Sale s
JOIN Dimension.Employee e ON s.[Salesperson Key] = e.[Employee Key]
GROUP BY e.Employee
ORDER BY Variance DESC;

(10 rows affected)

Total execution time: 00:00:04.586

Employee,NumberOfSales,EmployeeAverageProfit,CompanyAverageProfit,Variance
Archer Lamble,23331,383.785587,375.57,8.215587
Hudson Onslow,22681,382.361564,375.57,6.791564
Kayla Woodcock,23079,378.221296,375.57,2.651296
Lily Code,22642,376.732342,375.57,1.162342
Taj Shand,22975,374.907621,375.57,-0.662379
Jack Potter,22784,373.321341,375.57,-2.248659
Hudson Hollinworth,22902,373.274452,375.57,-2.295548
Sophia Hinton,22906,372.689317,375.57,-2.880683
Amy Trefl,22444,371.730703,375.57,-3.839297
Anthony Grosse,22521,368.361118,375.57,-7.208882


## <span style="color: var(--vscode-foreground);">Proposition 4: Customer Segmentation by Purchase Behavior (RFM)</span>

**Functional Specification:**

- **Objective:** To segment customers into 'High-Value', 'At-Risk', and 'New' categories based on Recency, Frequency, and Monetary value.
- **End-User:** Marketing and Customer Relationship Management (CRM) teams.
- **Business Value:** Enables highly targeted marketing campaigns: loyalty rewards for high-value customers, re-engagement campaigns for at-risk customers, and welcome offers for new ones, maximizing marketing ROI.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [4]:
WITH CustomerRFM AS (
    -- Step 1: Calculate Recency, Frequency, and Monetary values for each customer.
    SELECT
        c.Customer,
        MAX(s.[Invoice Date Key]) AS LastPurchaseDate,
        DATEDIFF(day, MAX(s.[Invoice Date Key]), GETDATE()) AS Recency,
        COUNT(DISTINCT s.[Sale Key]) AS Frequency,
        SUM(s.[Total Including Tax]) AS Monetary
    FROM Fact.Sale s
    JOIN Dimension.Customer c ON s.[Customer Key] = c.[Customer Key]
    GROUP BY c.Customer
)
-- Step 2: Assign a segment based on the calculated RFM values.
SELECT
    Customer,
    Recency,
    Frequency,
    Monetary,
    CASE
        WHEN Recency <= 60 AND Frequency >= 5 AND Monetary >= 5000 THEN 'High-Value Customer'
        WHEN Recency > 180 AND Frequency < 3 THEN 'At-Risk Customer'
        WHEN Recency <= 30 AND Frequency = 1 THEN 'New Customer'
        ELSE 'Regular Customer'
    END AS CustomerSegment
FROM CustomerRFM
ORDER BY Monetary DESC;

(403 rows affected)

Total execution time: 00:00:02.163

Customer,Recency,Frequency,Monetary,CustomerSegment
Unknown,3405,84297,73037043.78,Regular Customer
"Tailspin Toys (Inguadona, MN)",3412,434,438689.81,Regular Customer
"Tailspin Toys (Minidoka, ID)",3410,419,427445.57,Regular Customer
"Wingtip Toys (Sarversville, PA)",3430,432,420001.08,Regular Customer
"Tailspin Toys (Long Meadow, MD)",3406,392,414934.18,Regular Customer
"Wingtip Toys (San Jacinto, CA)",3412,398,408524.95,Regular Customer
"Wingtip Toys (Cuyamungue, NM)",3408,401,405824.67,Regular Customer
"Wingtip Toys (Morrison Bluff, AR)",3412,368,404945.65,Regular Customer
"Wingtip Toys (Grabill, IN)",3406,389,403635.05,Regular Customer
"Wingtip Toys (West Frostproof, FL)",3409,436,398563.46,Regular Customer


## <span style="color: var(--vscode-foreground);">Proposition 5: Top 5 Most Profitable Geographic Regions</span>

**Functional Specification:**

- **Objective:** To identify the top 5 most profitable cities and the top-performing salesperson within each of those cities.
- **End-User:** Sales Directors and Expansion Strategy teams.
- **Business Value:** Pinpoints not just \*where\* our most profitable markets are, but \*who\* is driving that success, informing decisions on sales territories and resource allocation.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [5]:
WITH CityProfit AS (
    -- Step 1: Calculate the total profit generated in each city and rank them.
    SELECT
        c.[City Key],
        c.[City],
        c.[State Province],
        SUM(s.Profit) AS TotalProfit,
        DENSE_RANK() OVER(ORDER BY SUM(s.Profit) DESC) AS ProfitRank
    FROM Fact.Sale s
    JOIN Dimension.City c ON s.[City Key] = c.[City Key]
    GROUP BY c.[City Key], c.[City], c.[State Province]
),
TopSalespersonInCity AS (
    -- Step 2: For each city, find the salesperson who generated the most profit.
    SELECT
        s.[City Key],
        e.Employee,
        SUM(s.Profit) AS SalespersonProfit,
        ROW_NUMBER() OVER(PARTITION BY s.[City Key] ORDER BY SUM(s.Profit) DESC) AS SalespersonRank
    FROM Fact.Sale s
    JOIN Dimension.Employee e ON s.[Salesperson Key] = e.[Employee Key]
    GROUP BY s.[City Key], e.Employee
)
-- Step 3: Join the top cities with their top salesperson.
SELECT
    cp.City,
    cp.[State Province],
    cp.TotalProfit,
    ts.Employee AS TopSalesperson
FROM CityProfit cp
JOIN TopSalespersonInCity ts ON cp.[City Key] = ts.[City Key]
WHERE cp.ProfitRank <= 5 AND ts.SalespersonRank = 1
ORDER BY cp.TotalProfit DESC;

(5 rows affected)

Total execution time: 00:00:02.004

City,State Province,TotalProfit,TopSalesperson
Sinclair,Wyoming,264147.10,Lily Code
Akhiok,Alaska,259554.30,Sophia Hinton
Cherry Grove Beach,South Carolina,257902.75,Hudson Hollinworth
East Fultonham,Ohio,244496.05,Anthony Grosse
Teutopolis,Illinois,234619.20,Taj Shand


## <span style="color: var(--vscode-foreground);">Proposition 6: Products with High Profit Margin but Low Sales Volume</span>

**Functional Specification:**

- **Objective:** To identify "hidden gem" products with a high average profit margin (over 40%) but low sales volume (fewer than 100 units sold).
- **End-User:** Product and Marketing Managers.
- **Business Value:** Uncovers highly profitable but undersold products. A targeted marketing campaign for these items could significantly boost company profits with minimal effort.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [6]:
WITH ProductPerformance AS (
    -- Step 1: Calculate sales volume and average profit margin for each product.
    SELECT
        si.[Stock Item],
        SUM(s.Quantity) AS TotalQuantitySold,
        AVG(s.Profit / NULLIF(s.[Total Excluding Tax], 0)) * 100 AS AverageProfitMargin
    FROM Fact.Sale s
    JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
    GROUP BY si.[Stock Item]
)
-- Step 2: Filter for products that meet the 'hidden gem' criteria.
SELECT
    [Stock Item],
    TotalQuantitySold,
    CAST(AverageProfitMargin AS DECIMAL(10, 2)) AS AverageProfitMargin
FROM ProductPerformance
WHERE
    AverageProfitMargin > 40  -- High profit margin
    AND TotalQuantitySold < 100 -- Low sales volume
ORDER BY AverageProfitMargin DESC;

(0 rows affected)

Total execution time: 00:00:01.211

Stock Item,TotalQuantitySold,AverageProfitMargin


## <span style="color: var(--vscode-foreground);">Proposition 7: Inventory Turnover Rate for Top 10 Products</span>

**Functional Specification:**

- **Objective:** To calculate the inventory turnover rate for the 10 products with the highest sales volume to assess stock management efficiency.
- **End-User:** Supply Chain and Finance departments.
- **Business Value:** Measures how efficiently we sell and replace inventory for our most popular items. This is a key indicator of operational efficiency and effective cash flow management.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [17]:
WITH CostOfGoodsSold AS (
    -- Step 1: Calculate the cost of goods sold (COGS) for each item.
    SELECT
        s.[Stock Item Key],
        SUM(s.Quantity * si.[Unit Price]) AS TotalCOGS
    FROM Fact.Sale s
    JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
    GROUP BY s.[Stock Item Key]
),
AverageInventory AS (
    -- Step 2: Calculate the average inventory value using the correct column name.
    SELECT
        sh.[Stock Item Key],
        AVG(sh.[Quantity On Hand] * si.[Unit Price]) AS AvgInventoryValue
    FROM Fact.[Stock Holding] sh
    JOIN Dimension.[Stock Item] si ON sh.[Stock Item Key] = si.[Stock Item Key]
    GROUP BY sh.[Stock Item Key]
)
-- Step 3: Join the data and calculate the turnover rate for the top 10 selling items.
SELECT TOP 10
    si.[Stock Item],
    cogs.TotalCOGS,
    ai.AvgInventoryValue,
    (cogs.TotalCOGS / NULLIF(ai.AvgInventoryValue, 0)) AS InventoryTurnoverRate
FROM CostOfGoodsSold cogs
JOIN AverageInventory ai ON cogs.[Stock Item Key] = ai.[Stock Item Key]
JOIN Dimension.[Stock Item] si ON cogs.[Stock Item Key] = si.[Stock Item Key]
WHERE ai.AvgInventoryValue IS NOT NULL
ORDER BY cogs.TotalCOGS DESC;

(0 rows affected)

Total execution time: 00:00:00.107

Stock Item,TotalCOGS,AvgInventoryValue,InventoryTurnoverRate


## <span style="color: var(--vscode-foreground);">Proposition 8: Order Fulfillment Efficiency (Order-to-Ship Time)</span>

**Functional Specification:**

- **Objective:** To analyze the average time (in hours) from order placement to shipment, segmented by state, to identify logistical bottlenecks.
- **End-User:** Operations and Logistics Managers.
- **Business Value:** Directly measures warehouse efficiency. Highlighting regional delays allows for targeted improvements to staffing, processes, or shipping partners, ultimately improving customer satisfaction.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [15]:
SELECT
    c.[State Province],
    COUNT(DISTINCT o.[Order Key]) AS NumberOfOrders,
    AVG(CAST(DATEDIFF(hour, o.[Order Date Key], o.[Picked Date Key]) AS DECIMAL(10,2))) AS AvgHoursToPick
FROM Fact.[Order] o
JOIN Dimension.City c ON o.[City Key] = c.[City Key]
WHERE
    o.[Picked Date Key] IS NOT NULL
    AND o.[Order Date Key] < o.[Picked Date Key]
GROUP BY
    c.[State Province]
HAVING
    COUNT(DISTINCT o.[Order Key]) > 50
ORDER BY
    AvgHoursToPick DESC;

(36 rows affected)

Total execution time: 00:00:00.988

State Province,NumberOfOrders,AvgHoursToPick
Indiana,87,2297.655172
Massachusetts[E],68,2151.176470
Missouri,136,2046.176470
Arizona,79,1892.050632
Michigan,94,1886.042553
North Carolina,124,1872.387096
Alaska,75,1821.440000
South Dakota,53,1816.301886
New York,218,1761.247706
California,251,1746.741035


## <span style="color: var(--vscode-foreground);">Proposition 9: Repeat Customer Rate Over Time</span>

**Functional Specification:**

- **Objective:** To calculate the monthly percentage of sales that come from repeat customers, measuring customer loyalty over time.
- **End-User:** Marketing and Executive Leadership.
- **Business Value:** Tracks the effectiveness of customer retention strategies. A rising repeat customer rate is a strong indicator of a healthy business and brand loyalty.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [10]:
WITH CustomerFirstPurchase AS (
    -- Step 1: Find the first purchase date for every customer.
    SELECT
        [Customer Key],
        MIN([Invoice Date Key]) AS FirstPurchaseDate
    FROM Fact.Sale
    GROUP BY [Customer Key]
),
MonthlySalesType AS (
    -- Step 2: Classify each sale as 'New' or 'Repeat'.
    SELECT
        FORMAT(s.[Invoice Date Key], 'yyyy-MM') AS SaleMonth,
        CASE
            WHEN s.[Invoice Date Key] = cfp.FirstPurchaseDate THEN 'New Customer'
            ELSE 'Repeat Customer'
        END AS CustomerType
    FROM Fact.Sale s
    JOIN CustomerFirstPurchase cfp ON s.[Customer Key] = cfp.[Customer Key]
)
-- Step 3: Pivot the data and calculate the monthly repeat customer rate.
SELECT
    SaleMonth,
    CAST(SUM(CASE WHEN CustomerType = 'Repeat Customer' THEN 1 ELSE 0 END) AS FLOAT) * 100 /
    CAST(COUNT(*) AS FLOAT) AS RepeatCustomerRate
FROM MonthlySalesType
GROUP BY SaleMonth
ORDER BY SaleMonth;

(41 rows affected)

Total execution time: 00:00:22.672

SaleMonth,RepeatCustomerRate
2013-01,75.44796035074343
2013-02,97.00566495818721
2013-03,99.45590994371481
2013-04,100
2013-05,100
2013-06,100
2013-07,100
2013-08,100
2013-09,100
2013-10,100


## <span style="color: var(--vscode-foreground);">Proposition 10: Correlating Purchase of Products (Market Basket Analysis)</span>

**Functional Specification:**

- **Objective:** For the most frequently sold product, find the top 5 other products most often purchased with it in the same transaction.
- **End-User:** Merchandising and Marketing teams.
- **Business Value:** This "Market Basket Analysis" provides powerful insights for cross-selling ("Customers who bought X also bought Y"), creating product bundles, and designing promotional offers.
- **AI Footnote:** Assistance was provided by Google's Gemini for brainstorming propositions, writing advanced SQL queries, and explaining the logic.

In [11]:
-- Step 1: Find the key for the most frequently sold product.
DECLARE @TopProductKey INT = (
    SELECT TOP 1 [Stock Item Key]
    FROM Fact.Sale
    GROUP BY [Stock Item Key]
    ORDER BY COUNT(*) DESC
);

-- Step 2: Find all orders containing that top product.
WITH OrdersWithTopProduct AS (
    SELECT DISTINCT [Sale Key]
    FROM Fact.Sale
    WHERE [Stock Item Key] = @TopProductKey
)
-- Step 3: Find all other products in those same orders and count them.
SELECT TOP 5
    si.[Stock Item] AS FrequentlyBoughtWithTopProduct,
    COUNT(*) AS PurchaseFrequency
FROM Fact.Sale s
JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
WHERE
    s.[Sale Key] IN (SELECT [Sale Key] FROM OrdersWithTopProduct)
    AND s.[Stock Item Key] != @TopProductKey
GROUP BY
    si.[Stock Item]
ORDER BY
    PurchaseFrequency DESC;

(0 rows affected)

Total execution time: 00:00:00.479

FrequentlyBoughtWithTopProduct,PurchaseFrequency
